In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import requests
from tabulate import tabulate

INPUT SYMBOL

In [ ]:
symbol = input("Inserte el ticker de la empresa: ").upper()

DESCARGA Y ORDENAMIENTO DE DATOS

In [ ]:
api_key = input("Inserte su API Key de Alpha Vantage: ")

INCOME_STATEMENT = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}'
CASH_FLOW = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={api_key}'
BALANCE_SHEET = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={api_key}'
PRICES = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={api_key}'

# Descargar los datos
r_INCOME_STATEMENT = requests.get(INCOME_STATEMENT)
data_INCOME_STATEMENT = r_INCOME_STATEMENT.json()

r_CASH_FLOW = requests.get(CASH_FLOW)
data_CASH_FLOW = r_CASH_FLOW.json()

r_BALANCE_SHEET = requests.get(BALANCE_SHEET)
data_BALANCE_SHEET = r_BALANCE_SHEET.json()

r_PRICES = requests.get(PRICES)
data_PRICES = r_PRICES.json()

In [ ]:
# Convertir los reportes a formato "tipo Excel" esperado por el resto del código
def json_to_excel_format(df, key='fiscalDateEnding'):
    df = df.copy()
    df[key] = pd.to_datetime(df[key])
    df = df.sort_values(key)
    df.set_index(key, inplace=True)
    df_T = df.transpose()
    # Opcional: dejar la fecha como string para igualar formato Excel
    df_T.columns = df_T.columns.strftime('%Y-%m-%d')
    # Primer columna (nombre del concepto) con el mismo formato que tus excels
    df_T.insert(0, df_T.index.name or 'Concepto', df_T.index)
    df_T.reset_index(drop=True, inplace=True)
    return df_T

# Income statement
if 'annualReports' in data_INCOME_STATEMENT:
    income_statement_df = pd.DataFrame(data_INCOME_STATEMENT['annualReports'])
    income_statement_df = json_to_excel_format(income_statement_df)
else:
    raise ValueError("No se encontraron datos de INCOME_STATEMENT para el símbolo dado.")

# Cash flow
if 'annualReports' in data_CASH_FLOW:
    cash_flow_df = pd.DataFrame(data_CASH_FLOW['annualReports'])
    cash_flow_df = json_to_excel_format(cash_flow_df)
else:
    raise ValueError("No se encontraron datos de CASH_FLOW para el símbolo dado.")

# Balance sheet
if 'annualReports' in data_BALANCE_SHEET:
    balance_sheet_df = pd.DataFrame(data_BALANCE_SHEET['annualReports'])
    balance_sheet_df = json_to_excel_format(balance_sheet_df)
else:
    raise ValueError("No se encontraron datos de BALANCE_SHEET para el símbolo dado.")

# Precios
if 'Time Series (Daily)' in data_PRICES:
    prices_df = pd.DataFrame.from_dict(data_PRICES['Time Series (Daily)'], orient='index')
    prices_df.index = pd.to_datetime(prices_df.index)
    prices_df = prices_df[['4. close']].rename(columns={'4. close': 'Close'})
    prices_df['Close'] = prices_df['Close'].astype(float)
    prices_df = prices_df.sort_index()
    prices_df.reset_index(inplace=True)
    prices_df.rename(columns={'index': 'Unnamed: 0'}, inplace=True)  # Para igualar el formato del excel
else:
    raise ValueError("No se encontraron datos de precios en la respuesta JSON")

COCIENTE CORRIENTE

In [ ]:
# Buscar las filas correspondientes a 'totalCurrentAssets' y 'totalCurrentLiabilities' en la segunda columna
total_current_assets = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalCurrentAssets']
total_current_liabilities = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalCurrentLiabilities']

# Asegurarse de que se encontró una única fila para cada uno
if total_current_assets.shape[0] != 1 or total_current_liabilities.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'totalCurrentAssets' o 'totalCurrentLiabilities'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
total_current_assets_values = total_current_assets.iloc[0, 1:].astype(float)
total_current_liabilities_values = total_current_liabilities.iloc[0, 1:].astype(float)

# Calcular el Cociente Corriente
Cociente_corriente = (total_current_liabilities_values / total_current_assets_values) * 100

# Crear un nuevo DataFrame para los ratios de liquidez en formato de porcentaje
ratios_liquidez_df = pd.DataFrame({
    'Fecha': total_current_assets.columns[1:],  # Excluyendo la primera columna
    'Cociente Corriente': Cociente_corriente
})

# Convertir los valores a formato de porcentaje
ratios_liquidez_df['Cociente Corriente'] = ratios_liquidez_df['Cociente Corriente'].map("{:.2f}%".format)

# Ajustar el índice del DataFrame para evitar duplicación de la columna Fecha
ratios_liquidez_df.set_index('Fecha', inplace=True)

RELACION DE PRUEBA ACIDA

In [ ]:
# Buscar las filas correspondientes a 'totalCurrentAssets', 'inventory' y 'totalCurrentLiabilities'
total_current_assets = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalCurrentAssets']
inventory = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'inventory']
total_current_liabilities = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalCurrentLiabilities']

# Asegurarse de que se encontró una única fila para cada uno
if total_current_assets.shape[0] != 1 or inventory.shape[0] != 1 or total_current_liabilities.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'totalCurrentAssets', 'inventory' o 'totalCurrentLiabilities'")

# Usar pd.to_numeric para convertir a float, manejando errores
total_current_assets_values = pd.to_numeric(total_current_assets.iloc[0, 1:], errors='coerce')
inventory_values = pd.to_numeric(inventory.iloc[0, 1:], errors='coerce')
total_current_liabilities_values = pd.to_numeric(total_current_liabilities.iloc[0, 1:], errors='coerce')

# Calcular la Relación de Prueba Ácida
relacion_prueba_acida = (total_current_assets_values - inventory_values) / total_current_liabilities_values

# Crear un DataFrame para la fecha y la Relación de Prueba Ácida
relacion_prueba_acida_df = pd.DataFrame({
    'Fecha': total_current_assets.columns[1:],  # Excluyendo la primera columna
    'Relación de Prueba Ácida': relacion_prueba_acida.values
})

# Si querés alinear por fecha (muy recomendado si hay riesgo de desorden), convertí fechas:
relacion_prueba_acida_df['Fecha'] = pd.to_datetime(relacion_prueba_acida_df['Fecha'], errors='coerce')
relacion_prueba_acida_df.set_index('Fecha', inplace=True)

# Ahora agregá la columna al DataFrame ratios_liquidez_df alineando por el índice
ratios_liquidez_df['Relación de Prueba Ácida'] = relacion_prueba_acida_df['Relación de Prueba Ácida']

COEFICIENTE DE EFECTIVO

In [ ]:
# Buscar las filas correspondientes a 'cashAndShortTermInvestments' y 'totalCurrentLiabilities' en la primera columna
cash_and_short_term_investments = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'cashAndShortTermInvestments']
total_current_liabilities = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalCurrentLiabilities']

# Asegurarse de que se encontró una única fila para cada uno
if cash_and_short_term_investments.shape[0] != 1 or total_current_liabilities.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'cashAndShortTermInvestments' o 'totalCurrentLiabilities'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
cash_and_short_term_investments_values = cash_and_short_term_investments.iloc[0, 1:].astype(float)
total_current_liabilities_values = total_current_liabilities.iloc[0, 1:].astype(float)

# Calcular el Coeficiente de Efectivo en porcentaje
coeficiente_efectivo = (cash_and_short_term_investments_values / total_current_liabilities_values) * 100

# Crear un DataFrame para la fecha y el Coeficiente de Efectivo
coeficiente_efectivo_df = pd.DataFrame({
    'Fecha': cash_and_short_term_investments.columns[1:],  # Excluyendo la primera columna
    'Coeficiente de Efectivo': coeficiente_efectivo
})

# Convertir los valores a formato de porcentaje
coeficiente_efectivo_df['Coeficiente de Efectivo'] = coeficiente_efectivo_df['Coeficiente de Efectivo'].map("{:.2f}%".format)

# Agregar la columna al DataFrame ratios_liquidez_df
ratios_liquidez_df['Coeficiente de Efectivo'] = coeficiente_efectivo_df['Coeficiente de Efectivo'].values

RATIO DE FLUJO DE CAJA OPERATIVO

In [ ]:
# Buscar las filas correspondientes a 'operatingCashflow' en cash_flow_df y 'totalCurrentLiabilities' en balance_sheet_df
operating_cashflow = cash_flow_df[cash_flow_df.iloc[:, 0] == 'operatingCashflow']
total_current_liabilities = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalCurrentLiabilities']

# Asegurarse de que se encontró una única fila para cada uno
if operating_cashflow.shape[0] != 1 or total_current_liabilities.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'operatingCashflow' o 'totalCurrentLiabilities'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
operating_cashflow_values = operating_cashflow.iloc[0, 1:].astype(float)
total_current_liabilities_values = total_current_liabilities.iloc[0, 1:].astype(float)

# Calcular el Ratio de Flujo de Caja Operativo en porcentaje
ratio_flujo_caja_operativo = (operating_cashflow_values / total_current_liabilities_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Flujo de Caja Operativo
ratio_flujo_caja_operativo_df = pd.DataFrame({
    'Fecha': operating_cashflow.columns[1:],  # Excluyendo la primera columna
    'Ratio de Flujo de Caja Operativo': ratio_flujo_caja_operativo
})

# Convertir los valores a formato de porcentaje
ratio_flujo_caja_operativo_df['Ratio de Flujo de Caja Operativo'] = ratio_flujo_caja_operativo_df['Ratio de Flujo de Caja Operativo'].map("{:.2f}%".format)

# Agregar la columna al DataFrame ratios_liquidez_df
ratios_liquidez_df['Ratio de Flujo de Caja Operativo'] = ratio_flujo_caja_operativo_df['Ratio de Flujo de Caja Operativo'].values

***RATIOS DE LIQUIDEZ***

In [ ]:
# Mostrar el DataFrame como tabla con bordes cuadrados (fancy_grid)
print("DataFrame con Ratios de Liquidez por Año:")
print(tabulate(ratios_liquidez_df, headers='keys', tablefmt='fancy_grid', showindex=True))

In [ ]:
# Convertir las columnas de porcentaje a valores numéricos, manejando excepciones
def convert_percentage(x):
    try:
        return float(x.strip('%'))
    except AttributeError:
        return x

ratios_liquidez_df = ratios_liquidez_df.applymap(convert_percentage)

# Graficar cada columna del DataFrame
for column in ratios_liquidez_df.columns:
    plt.figure(figsize=(10, 6))
    plt.plot(ratios_liquidez_df.index, ratios_liquidez_df[column], marker='o', linestyle='-', color='b')
    plt.title(f'{column} por Año')
    plt.xlabel('Fecha')
    plt.ylabel(column)
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

RATIO DE ENDEUDAMIENTO

In [ ]:
# Buscar las filas correspondientes a 'totalLiabilities' y 'totalAssets' en la primera columna
total_liabilities = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalLiabilities']
total_assets = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalAssets']

# Asegurarse de que se encontró una única fila para cada uno
if total_liabilities.shape[0] != 1 or total_assets.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'totalLiabilities' o 'totalAssets'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
total_liabilities_values = total_liabilities.iloc[0, 1:].astype(float)
total_assets_values = total_assets.iloc[0, 1:].astype(float)

# Calcular el Ratio de Endeudamiento en porcentaje
ratio_endeudamiento = (total_liabilities_values / total_assets_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Endeudamiento
apalancamiento_df = pd.DataFrame({
    'Fecha': total_liabilities.columns[1:],  # Excluyendo la primera columna
    'Ratio de Endeudamiento': ratio_endeudamiento
})

# Convertir los valores a formato de porcentaje
apalancamiento_df['Ratio de Endeudamiento'] = apalancamiento_df['Ratio de Endeudamiento'].map("{:.2f}%".format)

# Establecer la columna de fechas como índice para evitar duplicación
apalancamiento_df.set_index('Fecha', inplace=True)

RATIO DEUDA/CAPITAL

In [ ]:
# Buscar las filas correspondientes a 'totalLiabilities' y 'totalShareholderEquity' en la primera columna
total_liabilities = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalLiabilities']
total_shareholder_equity = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalShareholderEquity']

# Asegurarse de que se encontró una única fila para cada uno
if total_liabilities.shape[0] != 1 or total_shareholder_equity.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'totalLiabilities' o 'totalShareholderEquity'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
total_liabilities_values = total_liabilities.iloc[0, 1:].astype(float)
total_shareholder_equity_values = total_shareholder_equity.iloc[0, 1:].astype(float)

# Calcular el Ratio de Deuda a Capital
ratio_deuda_capital = (total_liabilities_values / total_shareholder_equity_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Deuda a Capital
ratio_deuda_capital_df = pd.DataFrame({
    'Fecha': total_liabilities.columns[1:],  # Excluyendo la primera columna
    'Ratio de Deuda a Capital': ratio_deuda_capital
})

# Convertir la columna 'Fecha' a tipo datetime
ratio_deuda_capital_df['Fecha'] = pd.to_datetime(ratio_deuda_capital_df['Fecha'])

# Establecer 'Fecha' como índice en ratio_deuda_capital_df
ratio_deuda_capital_df.set_index('Fecha', inplace=True)

# Si 'Fecha' está en las columnas de apalancamiento_df, eliminarla para evitar duplicaciones
if 'Fecha' in apalancamiento_df.columns:
    apalancamiento_df.drop(columns=['Fecha'], inplace=True)

# Asegurarse de que el índice de apalancamiento_df sea de tipo datetime para la columna 'Fecha'
if not pd.api.types.is_datetime64_any_dtype(apalancamiento_df.index):
    apalancamiento_df.index = pd.to_datetime(apalancamiento_df.index)

# Agregar la columna al DataFrame apalancamiento_df
apalancamiento_df['Ratio de Deuda a Capital'] = ratio_deuda_capital_df['Ratio de Deuda a Capital']

RATIO EBIT/INTERESES

In [ ]:
# Buscar las filas correspondientes a 'ebit' y 'interestExpense' en la primera columna
ebit = income_statement_df[income_statement_df.iloc[:, 0] == 'ebit']
interest_expense = income_statement_df[income_statement_df.iloc[:, 0] == 'interestExpense']

# Asegurarse de que se encontró una única fila para cada uno
if ebit.shape[0] != 1 or interest_expense.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'ebit' o 'interestExpense'")

# Obtener los valores excluyendo la primera columna (que contiene el nombre de las filas)
ebit_values = ebit.iloc[0, 1:].astype(float).values
interest_expense_values = interest_expense.iloc[0, 1:].astype(float).values

# Calcular el RATIO EBIT/INTERESES
ratio_ebit_intereses = ebit_values / interest_expense_values

# Crear un DataFrame para la fecha y el RATIO EBIT/INTERESES
ratio_ebit_intereses_df = pd.DataFrame({
    'Fecha': ebit.columns[1:],  # Excluyendo la primera columna
    'RATIO EBIT/INTERESES': ratio_ebit_intereses
})

# Convertir la columna 'Fecha' a tipo datetime
ratio_ebit_intereses_df['Fecha'] = pd.to_datetime(ratio_ebit_intereses_df['Fecha'])

# Establecer 'Fecha' como índice en ratio_ebit_intereses_df
ratio_ebit_intereses_df.set_index('Fecha', inplace=True)

# Si 'Fecha' está en las columnas de apalancamiento_df, eliminarla para evitar duplicaciones
if 'Fecha' in apalancamiento_df.columns:
    apalancamiento_df.drop(columns=['Fecha'], inplace=True)

# Asegurarse de que el índice de apalancamiento_df sea de tipo datetime para la columna 'Fecha'
if not pd.api.types.is_datetime64_any_dtype(apalancamiento_df.index):
    apalancamiento_df.index = pd.to_datetime(apalancamiento_df.index)

# Agregar la columna al DataFrame apalancamiento_df
apalancamiento_df['RATIO EBIT/INTERESES'] = ratio_ebit_intereses_df['RATIO EBIT/INTERESES']

***RATIOS DE APALANCAMIENTO***

In [ ]:
print("DataFrame con Ratios de Apalancamiento por Año:")
print(tabulate(apalancamiento_df, headers="keys", tablefmt="fancy_grid", showindex=True))

In [ ]:
# Verificar si 'Fecha' está en las columnas de apalancamiento_df
if 'Fecha' in apalancamiento_df.columns:
    # Convertir la columna 'Fecha' a tipo datetime si no lo está
    apalancamiento_df['Fecha'] = pd.to_datetime(apalancamiento_df['Fecha'])
    # Establecer 'Fecha' como índice
    apalancamiento_df.set_index('Fecha', inplace=True)

# Convertir las columnas de porcentaje a float para ordenarlas correctamente, si no están ya en float
if apalancamiento_df['Ratio de Endeudamiento'].dtype == 'object':
    apalancamiento_df['Ratio de Endeudamiento'] = apalancamiento_df['Ratio de Endeudamiento'].str.rstrip('%').astype('float') / 100
if apalancamiento_df['Ratio de Deuda a Capital'].dtype == 'object':
    apalancamiento_df['Ratio de Deuda a Capital'] = apalancamiento_df['Ratio de Deuda a Capital'].str.rstrip('%').astype('float') / 100

# Crear gráficos para cada ratio en el DataFrame apalancamiento_df
fig, axs = plt.subplots(len(apalancamiento_df.columns), 1, figsize=(10, 15))

for i, column in enumerate(apalancamiento_df.columns):
    axs[i].plot(apalancamiento_df.index, apalancamiento_df[column], marker='o')
    axs[i].set_title(column)
    axs[i].set_xlabel('Fecha')
    axs[i].set_ylabel(column)
    axs[i].grid(True)

plt.tight_layout()
plt.show()

MARGEN BRUTO

In [ ]:
# Buscar las filas correspondientes a 'grossProfit' y 'totalRevenue' en la primera columna
gross_profit = income_statement_df[income_statement_df.iloc[:, 0] == 'grossProfit']
total_revenue = income_statement_df[income_statement_df.iloc[:, 0] == 'totalRevenue']

# Asegurarse de que se encontró una única fila para cada uno
if gross_profit.shape[0] != 1 or total_revenue.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'grossProfit' o 'totalRevenue'")

# Convertir las columnas a tipo float excluyendo la primera columna (que contiene el nombre de las filas)
gross_profit_values = gross_profit.iloc[0, 1:].astype(float).values
total_revenue_values = total_revenue.iloc[0, 1:].astype(float).values

# Calcular el Ratio de Margen Bruto
ratio_margen_bruto = (gross_profit_values / total_revenue_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Margen Bruto
ratio_margen_bruto_df = pd.DataFrame({
    'Fecha': gross_profit.columns[1:],  # Excluyendo la primera columna
    'Ratio de Margen Bruto': ratio_margen_bruto
})

# Convertir la columna 'Fecha' a tipo datetime
ratio_margen_bruto_df['Fecha'] = pd.to_datetime(ratio_margen_bruto_df['Fecha'])

# Establecer 'Fecha' como índice en ratio_margen_bruto_df
ratio_margen_bruto_df.set_index('Fecha', inplace=True)

# Crear el DataFrame rentabilidad_df
rentabilidad_df = ratio_margen_bruto_df.copy()

# Convertir los valores de 'Ratio de Margen Bruto' a float sin el formato de porcentaje
rentabilidad_df['Ratio de Margen Bruto'] = rentabilidad_df['Ratio de Margen Bruto'].astype(float)

MARGEN OPERATIVO

In [ ]:
# Buscar las filas correspondientes a 'operatingIncome' y 'totalRevenue' en la primera columna
operating_income = income_statement_df[income_statement_df.iloc[:, 0] == 'operatingIncome']
total_revenue = income_statement_df[income_statement_df.iloc[:, 0] == 'totalRevenue']

# Asegurarse de que se encontró una única fila para cada uno
if operating_income.shape[0] != 1 or total_revenue.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'operatingIncome' o 'totalRevenue'")

# Convertir las columnas a tipo float excluyendo la primera columna (que contiene el nombre de las filas)
operating_income_values = operating_income.iloc[0, 1:].astype(float).values
total_revenue_values = total_revenue.iloc[0, 1:].astype(float).values

# Calcular el Ratio de Margen Operativo
ratio_margen_operativo = (operating_income_values / total_revenue_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Margen Operativo
ratio_margen_operativo_df = pd.DataFrame({
    'Fecha': operating_income.columns[1:],  # Excluyendo la primera columna
    'Ratio de Margen Operativo': ratio_margen_operativo
})

# Convertir la columna 'Fecha' a tipo datetime
ratio_margen_operativo_df['Fecha'] = pd.to_datetime(ratio_margen_operativo_df['Fecha'])

# Establecer 'Fecha' como índice en ratio_margen_operativo_df
ratio_margen_operativo_df.set_index('Fecha', inplace=True)

# Agregar esta columna a rentabilidad_df
if 'rentabilidad_df' in globals():
    rentabilidad_df = rentabilidad_df.drop(columns=['Ratio de Margen Operativo'], errors='ignore')
    rentabilidad_df = rentabilidad_df.join(ratio_margen_operativo_df, how='outer')
else:
    rentabilidad_df = ratio_margen_operativo_df.copy()

ROA

In [ ]:
# Buscar las filas correspondientes a 'totalRevenue', 'totalAssets' y 'netIncome'
total_revenue = income_statement_df[income_statement_df.iloc[:, 0] == 'totalRevenue']
total_assets = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalAssets']

# netIncome suele estar en cash_flow_df, pero puede estar en income_statement_df también:
if 'netIncome' in income_statement_df.iloc[:, 0].values:
    net_income = income_statement_df[income_statement_df.iloc[:, 0] == 'netIncome']
else:
    net_income = cash_flow_df[cash_flow_df.iloc[:, 0] == 'netIncome']

# Asegurarse de que se encontró una única fila para cada uno
if total_revenue.shape[0] != 1 or total_assets.shape[0] != 1 or net_income.shape[0] != 1:
    raise ValueError("No se encontraron filas únicas para 'totalRevenue', 'totalAssets' o 'netIncome'")

# Convertir columnas a float (excluyendo la primera columna)
fecha_cols = total_revenue.columns[1:]  # Todas salvo la primera

total_revenue_values = pd.to_numeric(total_revenue.iloc[0, 1:], errors='coerce').values
total_assets_values = pd.to_numeric(total_assets.iloc[0, 1:], errors='coerce').values
net_income_values = pd.to_numeric(net_income.iloc[0, 1:], errors='coerce').values

# Calcular el Ratio de Rentabilidad de los Activos (ROA)
ratio_rentabilidad_activos = (net_income_values / total_assets_values) * 100

# Crear DataFrame con fechas y ROA
ratio_rentabilidad_activos_df = pd.DataFrame({
    'Fecha': fecha_cols,
    'ROA': ratio_rentabilidad_activos
})

# Convertir 'Fecha' a datetime
ratio_rentabilidad_activos_df['Fecha'] = pd.to_datetime(ratio_rentabilidad_activos_df['Fecha'], errors='coerce')

# Establecer 'Fecha' como índice
ratio_rentabilidad_activos_df.set_index('Fecha', inplace=True)

# Agregar esta columna a rentabilidad_df (alineando por índice)
if 'rentabilidad_df' in globals():
    rentabilidad_df = rentabilidad_df.drop(columns=['ROA'], errors='ignore')
    rentabilidad_df = rentabilidad_df.join(ratio_rentabilidad_activos_df, how='outer')
else:
    rentabilidad_df = ratio_rentabilidad_activos_df.copy()

ROE

In [ ]:
# Buscar las filas correspondientes a 'netIncome' en income_statement_df y 'totalShareholderEquity' en balance_sheet_df
net_income = income_statement_df[income_statement_df.iloc[:, 0] == 'netIncome']
total_shareholder_equity = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalShareholderEquity']

# Asegurarse de que se encontró al menos una fila para cada uno
if net_income.shape[0] == 0 or total_shareholder_equity.shape[0] == 0:
    raise ValueError("No se encontraron filas para 'netIncome' o 'totalShareholderEquity'")

# Convertir las columnas a tipo float excluyendo la primera columna (que contiene el nombre de las filas)
net_income_values = net_income.iloc[0, 1:].astype(float).values
total_shareholder_equity_values = total_shareholder_equity.iloc[0, 1:].astype(float).values

# Calcular el Ratio de Rentabilidad sobre Fondos Propios (ROE)
ratio_rentabilidad_fondos_propios = (net_income_values / total_shareholder_equity_values) * 100

# Crear un DataFrame para la fecha y el Ratio de Rentabilidad sobre Fondos Propios (ROE)
ratio_rentabilidad_fondos_propios_df = pd.DataFrame({
    'Fecha': net_income.columns[1:],  # Excluyendo la primera columna
    'ROE': ratio_rentabilidad_fondos_propios
})

# Convertir la columna 'Fecha' a tipo datetime
ratio_rentabilidad_fondos_propios_df['Fecha'] = pd.to_datetime(ratio_rentabilidad_fondos_propios_df['Fecha'])

# Establecer 'Fecha' como índice en ratio_rentabilidad_fondos_propios_df
ratio_rentabilidad_fondos_propios_df.set_index('Fecha', inplace=True)

# Agregar esta columna a rentabilidad_df
if 'rentabilidad_df' in globals():
    rentabilidad_df = rentabilidad_df.drop(columns=['ROE'], errors='ignore')
    rentabilidad_df = rentabilidad_df.join(ratio_rentabilidad_fondos_propios_df, how='outer')
else:
    rentabilidad_df = ratio_rentabilidad_fondos_propios_df.copy()

P/B RATIO

In [ ]:
# Extraer las fechas (todas las columnas menos la primera, que es 'Concepto')
fecha_cols = balance_sheet_df.columns[1:]

# Encontrar el índice de las filas relevantes
idx_equity = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'totalShareholderEquity'].index[0]
idx_shares = balance_sheet_df[balance_sheet_df.iloc[:, 0] == 'commonStockSharesOutstanding'].index[0]

# Calcular Book Value Per Share para cada fecha
book_value_per_share = []
for fecha in fecha_cols:
    try:
        equity = float(balance_sheet_df.loc[idx_equity, fecha])
        shares = float(balance_sheet_df.loc[idx_shares, fecha])
        bvps = equity / shares if shares != 0 else float('nan')
    except Exception:
        bvps = float('nan')
    book_value_per_share.append(bvps)

# Encontrar el precio más cercano para cada fecha de reporte
closest_price = []
for fecha in fecha_cols:
    try:
        fecha_dt = pd.to_datetime(fecha)
        price_row = prices_df.iloc[(prices_df['Unnamed: 0']-fecha_dt).abs().argsort()[:1]]
        price = price_row['Close'].values[0]
    except Exception:
        price = float('nan')
    closest_price.append(price)

# Calcular el P/B Ratio
pb_ratio = [p/b if b and not pd.isna(b) and b != 0 else float('nan') for p, b in zip(closest_price, book_value_per_share)]

# Agregar el P/B Ratio a rentabilidad_df, alineando fechas
rentabilidad_df['P/B Ratio'] = pb_ratio

RENTABILIDAD POR DIVIDENDO

In [ ]:
# Encontrar la fila que corresponde a dividendPayoutCommonStock (puede que haya un label NaT en el índice)
mask = cash_flow_df.iloc[:, 0] == 'dividendPayoutCommonStock'
if mask.any():
    idx = cash_flow_df.index[mask][0]
    dividendos = pd.to_numeric(cash_flow_df.loc[idx, cash_flow_df.columns[1:]], errors='coerce')
else:
    dividendos = pd.Series([float('nan')] * (cash_flow_df.shape[1] - 1), index=cash_flow_df.columns[1:])

# Igual para commonStockSharesOutstanding en balance_sheet_df
mask_shares = balance_sheet_df.iloc[:, 0] == 'commonStockSharesOutstanding'
if mask_shares.any():
    idx_sh = balance_sheet_df.index[mask_shares][0]
    acciones = pd.to_numeric(balance_sheet_df.loc[idx_sh, balance_sheet_df.columns[1:]], errors='coerce')
else:
    acciones = pd.Series([float('nan')] * (balance_sheet_df.shape[1] - 1), index=balance_sheet_df.columns[1:])

# Dividendo por acción
dividendo_x_accion = dividendos / acciones

# Precio más cercano a la fecha de reporte
precios = []
for fecha in cash_flow_df.columns[1:]:
    try:
        fecha_dt = pd.to_datetime(fecha)
        price_row = prices_df.iloc[(prices_df['Unnamed: 0']-fecha_dt).abs().argsort()[:1]]
        price = price_row['Close'].values[0]
    except Exception:
        price = float('nan')
    precios.append(price)
precios_series = pd.Series(precios, index=cash_flow_df.columns[1:])

# Dividend yield (%)
dividend_yield = (dividendo_x_accion / precios_series) * 100

# Lo agregás al rentabilidad_df, alineando por índice/fechas en columnas
# Si el índice de rentabilidad_df son fechas, asegurate que coincidan exactamente en formato con cash_flow_df.columns[1:]
dividend_yield.index = pd.to_datetime(dividend_yield.index)
if not isinstance(rentabilidad_df.index, pd.DatetimeIndex):
    rentabilidad_df.index = pd.to_datetime(rentabilidad_df.index)

# Asignar alineando índices (fechas)
rentabilidad_df['Dividend Yield (%)'] = dividend_yield

EPS

In [ ]:
# 1. Las columnas son strings (e.g. '2005-09-30')
fecha_cols = cash_flow_df.columns[1:]  # O balance_sheet_df.columns[1:], ambas strings

# 2. Buscar la fila de netIncome
mask_net_income = cash_flow_df.iloc[:, 0] == 'netIncome'
if mask_net_income.any():
    idx_net = cash_flow_df.index[mask_net_income][0]
    net_income = pd.to_numeric(
        cash_flow_df.loc[idx_net, fecha_cols], errors='coerce'
    )
else:
    net_income = pd.Series([float('nan')]*len(fecha_cols), index=fecha_cols)

# 3. Buscar la fila de commonStockSharesOutstanding en balance_sheet_df
mask_shares = balance_sheet_df.iloc[:, 0] == 'commonStockSharesOutstanding'
if mask_shares.any():
    idx_shares = balance_sheet_df.index[mask_shares][0]
    acciones = pd.to_numeric(
        balance_sheet_df.loc[idx_shares, fecha_cols], errors='coerce'
    )
else:
    acciones = pd.Series([float('nan')]*len(fecha_cols), index=fecha_cols)

# 4. Calcular EPS por fecha
eps = net_income / acciones

# 5. Convertir índice de EPS y rentabilidad_df a datetime para alineación
eps.index = pd.to_datetime(eps.index)
if not isinstance(rentabilidad_df.index, pd.DatetimeIndex):
    rentabilidad_df.index = pd.to_datetime(rentabilidad_df.index)

# 6. Limpiar duplicados y NaT
eps = eps[~eps.index.duplicated(keep='first')]
eps = eps[~eps.index.isna()]
rentabilidad_df = rentabilidad_df[~rentabilidad_df.index.duplicated(keep='first')]
rentabilidad_df = rentabilidad_df[~rentabilidad_df.index.isna()]

# 7. Asignar EPS alineando por fecha
rentabilidad_df['EPS'] = eps

P/E RATIO

In [ ]:
# Renombrar la columna sin nombre en prices_df a 'date'
prices_df.rename(columns={'Unnamed: 0': 'date', 'Close': 'price'}, inplace=True)
prices_df['date'] = pd.to_datetime(prices_df['date'])

# Convertir la columna 'fiscalDateEnding' a tipo datetime en rentabilidad_df
rentabilidad_df.index = pd.to_datetime(rentabilidad_df.index, errors='coerce')

# Función para encontrar el precio más cercano a una fecha dada
def find_closest_price(date, prices_df):
    closest_date = prices_df.iloc[(prices_df['date'] - date).abs().argsort()[:1]]['date'].values[0]
    closest_price = prices_df[prices_df['date'] == closest_date]['price'].values[0]
    return closest_price

# Aplicar la función para encontrar el precio más cercano para cada fecha en rentabilidad_df
rentabilidad_df['closest_price'] = rentabilidad_df.index.to_series().apply(lambda x: find_closest_price(x, prices_df))

# Calcular el P/E Ratio
rentabilidad_df['P/E Ratio'] = rentabilidad_df['closest_price'] / rentabilidad_df['EPS']

# Eliminar la columna 'closest_price'
rentabilidad_df.drop(columns=['closest_price'], inplace=True)

***RATIOS DE RENTABILIDAD Y PRECIO***

In [ ]:
print("DataFrame con el Ratio de Rentabilidad sobre Fondos Propios por Año:")
print(tabulate(rentabilidad_df, headers="keys", tablefmt="fancy_grid", showindex=True))

In [ ]:
# Asegúrate de que rentabilidad_df tiene el índice de tipo datetime
rentabilidad_df.index = pd.to_datetime(rentabilidad_df.index)

# Ordenar el DataFrame por fechas
rentabilidad_df = rentabilidad_df.sort_index()

# Función para graficar cada columna con eje Y comenzando en 0
def graficar_columna(df, columna):
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df[columna], marker='o')
    plt.title(f'Gráfico de {columna}')
    plt.xlabel('Fecha')
    plt.ylabel(columna)
    plt.ylim(bottom=0)  # Establecer el límite inferior del eje Y a 0
    plt.grid(True)
    plt.show()

# Graficar cada columna del DataFrame
for columna in rentabilidad_df.columns:
    graficar_columna(rentabilidad_df, columna)

***GRÁFICO DE PRECIOS***

In [ ]:
# Asegúrate de que la columna 'date' esté en formato datetime
prices_df['date'] = pd.to_datetime(prices_df['date'])

# Establecer 'date' como índice
prices_df.set_index('date', inplace=True)

# Ordenar el DataFrame por fechas
prices_df = prices_df.sort_index()

# Graficar el DataFrame sin puntos, solo líneas
plt.figure(figsize=(12, 6))
plt.plot(prices_df.index, prices_df['price'])
plt.title('Gráfico de Precios')
plt.xlabel('Fecha')
plt.ylabel('Precio')
plt.ylim(bottom=0)  # Establecer el límite inferior del eje Y a 0
plt.grid(True)
plt.show()